<a href="https://colab.research.google.com/github/hectorpilo/bootcamp-ds-sonda/blob/main/Ejercicio_de_Regresi%C3%B3n_1_(Core).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Usamos la biblioteca google para poder usar archivos en nuestro drive.
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
path = "/content/drive/MyDrive/BBDD/house-prices.csv"
df = pd.read_csv(path)


#para limpiar cuando hay espacios en los nombres de las columnas. df.columns = df.columns.str.strip()


Inspección de la DATA

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()

Las Variables Brick y Neighborhood son categoricas, así que habrá que transformaras.

In [ ]:
df.describe().T

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().sum()

# **ANALISIS GRAFICO**

In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(x=df["Price"])

plt.title("Boxplot de Price")
plt.xlabel("Price")

plt.show()


In [ ]:
# 1. Calculamos los cuartiles
Q1 = df['Price'].quantile(0.25)
Q3 = df['Price'].quantile(0.75)
IQR = Q3 - Q1

# 2. Calculamos los límites para outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"Límite inferior: {lower_bound}")
print(f"Límite superior: {upper_bound}")

# 3. Filtramos los registros que son outliers
outliers = df[(df['Price'] < lower_bound) | (df['Price'] > upper_bound)]

print("Outliers encontrados:")
print(outliers)


# COMENTARIO: Hay un outlier, veremos como afecta eso nuestro modelo, haciendo un aprendizaje con el registro outlier y sin el.

Con el Outlier.

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
X = df.drop(columns=["Home", "Price", "Brick", "Neighborhood","Offers"]) #Por que HOME es un ID y me aportará ruido, y en el enunciado del ejercicio solo piden las columnas que se quedarán.
y = df["Price"]

In [ ]:
from sklearn.model_selection import train_test_split

# Dividir los datos: 80% entrenamiento, 20% prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.linear_model import LinearRegression

# Crear el modelo
modelo = LinearRegression()

# Ajustar el modelo con los datos de entrenamiento
modelo.fit(X_train, y_train)

In [ ]:
y_pred = modelo.predict(X_test)
y_pred

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Calculamos RMSE y R²
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.2f}")

# **ADICIONAL: Guardaré el modelo y haré unas pruebas.**

In [ ]:
import joblib

# Guardar el modelo entrenado en un archivo .pkl
joblib.dump(modelo, "modelo_regresion.pkl")


In [ ]:
nueva_casa = pd.DataFrame({
    "SqFt": [2000],
    "Bedrooms": [4],
    "Bathrooms": [3]
})

In [ ]:
import joblib

# Cargar el modelo desde el archivo .pkl
modelo_cargado = joblib.load("modelo_regresion.pkl")

# usarlo como antes
nueva_prediccion = modelo_cargado.predict(nueva_casa)
nueva_prediccion

In [ ]:
print(f"Precio estimado para la casa: ${nueva_prediccion[0]:,.2f}") #formateamos la salida para que se vea bonito.


# **Mas Pruebas**
Ahora, le pediré al usuario que ingrese los valores de los metros cuadrados, las habitaciones, los baños y de Offers.

In [58]:
import pandas as pd

# Pedir al usuario los valores
mts = float(input("Ingrese el tamaño en metros cuadrados (Mts²): "))
bedrooms = int(input("Ingrese el número de habitaciones (Bedrooms): "))
bathrooms = int(input("Ingrese el número de baños (Bathrooms): "))

# Convertir metros cuadrados a pies cuadrados
sqft = mts * 10.7639

# Crear el DataFrame solo con las columnas necesarias
nueva_casa = pd.DataFrame({
    "SqFt": [sqft],
    "Bedrooms": [bedrooms],
    "Bathrooms": [bathrooms]
})

# Hacer la predicción
prediccion = modelo.predict(nueva_casa)

# Mostrar el resultado formateado
print(f"\nPrecio estimado para la casa: ${prediccion[0]:,.2f}")



Ingrese el tamaño en metros cuadrados (Mts²): 150
Ingrese el número de habitaciones (Bedrooms): 3
Ingrese el número de baños (Bathrooms): 4

Precio estimado para la casa: $133,743.27
